## 무림페이퍼 진주공장 스팀수요 예측

#### 0) Import Lib

In [8]:
import pandas as pd
import joblib
import sys
sys.path.append('./' )    # 모듈 저장 위치 지정 
print(sys.path)
import FEMS_Moolim_SteamUsage_Predict_Module as MM

import pymysql

#import xgboost as xgb
#from xgboost import XGBRegressor

# from pandas import MultiIndex, Int64Index

['c:\\Users\\limjh\\OneDrive\\바탕 화면\\git_jhlim\\2. 프로젝트\\3. 무림제지FEMS', 'c:\\Users\\limjh\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip', 'c:\\Users\\limjh\\AppData\\Local\\Programs\\Python\\Python39\\DLLs', 'c:\\Users\\limjh\\AppData\\Local\\Programs\\Python\\Python39\\lib', 'c:\\Users\\limjh\\AppData\\Local\\Programs\\Python\\Python39', '', 'C:\\Users\\limjh\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\limjh\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32', 'C:\\Users\\limjh\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib', 'C:\\Users\\limjh\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin', 'c:\\Users\\limjh\\AppData\\Local\\Programs\\Python\\Python39\\lib\\site-packages', 'c:\\users\\limjh\\onedrive\\바탕 화면\\git_jhlim\\mlflow\\tests\\resources\\mlflow-test-plugin', './', './']


#### 1) Data Load

##### Sample

In [28]:
# 임시로 생성한 raw

# raw = pd.read_csv('./data/mart/03.pre-processed_mart_v2.csv', encoding='cp949')
# raw = raw.head(1)
# raw = raw.drop('PM3-S3SC.opr:me', axis=1)
# raw['DateTime'] = pd.to_datetime(raw['DateTime'])
# raw

# 아래와 같은 형태로 데이터 수집이 필요합니다.
# DateTime, STEAM, 37개 영향인자(X)
# 15분간 축적된 데이터를 기반으로 평균(대표값)이 산출된 (1, 39) 크기의 데이터프레임
# 해당 STEAM 값은 15분 전 예측된 값과 비교하기 위한 실제 스팀사용량 입니다.

,DateTime,STEAM,320FIA7505:av,320LIC7525:con,320LIC7526:con,320LIC7527:con,320LIC7527:me,320LIC7528:con,320LIC7537:con,320LIC7537:me,...,PM3_MOISTURE_PV,PM3_MOISTURE_PV2,PM3_PREDRYER_OP,PM3_PREDRYER_PV,PM3_STOCK_CONSISTENCY_PV,PM3_STOCK_FLOW_PV,PM3_STOCK_VALVE_P_PV,PM3_AFTERDRYER_PV,320PIC7514:me,320PIC7513:me
0,2022-01-01 00:15:00,8.84961,34.083571,52.616154,60.915455,11.093333,49.274286,31.770769,3.779231,46.901333,...,6.216,5.0342,84.36,4.798333,4.165667,10.621533,64.077333,1.907533,2.679231,1.339333


##### DB 연결

In [9]:
# Connect to MariaDB by pymysql
try:
    conn = pymysql.connect(
        user="fcstuser",
        password="fcstuser1!",
        host="ap.kfems.net",
        port=13306,
        database="fems"
    )
    print("Connect Success")
    
except pymysql.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()


Connect Success


##### 데이터 조회

In [10]:
# 조회 query
select_query = """SELECT DateTime
      , STEAM
      , `320FIA7505:av`
      , `320LIC7525:con`
      , `320LIC7526:con`
      , `320LIC7527:con`
      , `320LIC7527:me`
      , `320LIC7528:con`
      , `320LIC7537:con`
      , `320LIC7537:me`
      , `320PIC7509:me`
      , `320PIC7510:con`
      , `320PIC7510:me`
      , `320PIC7511:con`
      , `320PIC7511:me`
      , `320PIC7533:con`
      , `320PIC7533:me`
      , `320PIC7544:con`
      , `320PIC7544:me`
      , `320TIC7540:me`
      , `320TIC7545:con`
      , `320TIC7545:me`
      , `PM3_AFTERDRYER_OP`
      , `PM3_ASH_FLOW_OP`
      , `PM3_ASH_FLOW_PV`
      , `PM3_ASH_PV`
      , `PM3_BASIS_WEIGHT_PV`
      , `PM3_BASIS_WEIGHT_PV2`
      , `PM3_CW_PV`
      , `PM3_MOISTURE_PV`
      , `PM3_MOISTURE_PV2`
      , `PM3_PREDRYER_OP`
      , `PM3_PREDRYER_PV`
      , `PM3_STOCK_CONSISTENCY_PV`
      , `PM3_STOCK_FLOW_PV`
      , `PM3_STOCK_VALVE_P_PV`
      , `PM3_AFTERDRYER_PV`
      , `320PIC7514:me`
      , `320PIC7513:me`
  FROM (
         SELECT READ_DTTM AS DateTime
              , GROUP_CONCAT(IF(TAG_ID = 'STEAM', READ_VAL, NULL))                      AS STEAM
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7525:con' , READ_VAL, NULL))            AS `320LIC7525:con`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7526:con' , READ_VAL, NULL))            AS `320LIC7526:con`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7527:con' , READ_VAL, NULL))            AS `320LIC7527:con`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7527:me' , READ_VAL, NULL))             AS `320LIC7527:me`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7528:con' , READ_VAL, NULL))            AS `320LIC7528:con`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7537:con' , READ_VAL, NULL))            AS `320LIC7537:con`
              , GROUP_CONCAT(IF(TAG_ID = '320LIC7537:me' , READ_VAL, NULL))             AS `320LIC7537:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7509:me' , READ_VAL, NULL))             AS `320PIC7509:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7510:con' , READ_VAL, NULL))            AS `320PIC7510:con`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7510:me' , READ_VAL, NULL))             AS `320PIC7510:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7511:con' , READ_VAL, NULL))            AS `320PIC7511:con`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7511:me' , READ_VAL, NULL))             AS `320PIC7511:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7533:con' , READ_VAL, NULL))            AS `320PIC7533:con`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7533:me' , READ_VAL, NULL))             AS `320PIC7533:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7544:con' , READ_VAL, NULL))            AS `320PIC7544:con`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7544:me' , READ_VAL, NULL))             AS `320PIC7544:me`
              , GROUP_CONCAT(IF(TAG_ID = '320TIC7540:me' , READ_VAL, NULL))             AS `320TIC7540:me`
              , GROUP_CONCAT(IF(TAG_ID = '320TIC7545:con' , READ_VAL, NULL))            AS `320TIC7545:con`
              , GROUP_CONCAT(IF(TAG_ID = '320TIC7545:me' , READ_VAL, NULL))             AS `320TIC7545:me`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_AFTERDRYER_OP' , READ_VAL, NULL))         AS `PM3_AFTERDRYER_OP`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_ASH_FLOW_OP' , READ_VAL, NULL))           AS `PM3_ASH_FLOW_OP`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_ASH_FLOW_PV' , READ_VAL, NULL))           AS `PM3_ASH_FLOW_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_ASH_PV' , READ_VAL, NULL))                AS `PM3_ASH_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_BASIS_WEIGHT_PV' , READ_VAL, NULL))       AS `PM3_BASIS_WEIGHT_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_BASIS_WEIGHT_PV2' , READ_VAL, NULL))      AS `PM3_BASIS_WEIGHT_PV2`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_CW_PV' , READ_VAL, NULL))                 AS `PM3_CW_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_MOISTURE_PV' , READ_VAL, NULL))           AS `PM3_MOISTURE_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_MOISTURE_PV2' , READ_VAL, NULL))          AS `PM3_MOISTURE_PV2`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_PREDRYER_OP' , READ_VAL, NULL))           AS `PM3_PREDRYER_OP`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_PREDRYER_PV' , READ_VAL, NULL))           AS `PM3_PREDRYER_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_STOCK_CONSISTENCY_PV' , READ_VAL, NULL))  AS `PM3_STOCK_CONSISTENCY_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_STOCK_FLOW_PV' , READ_VAL, NULL))         AS `PM3_STOCK_FLOW_PV`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_STOCK_VALVE_P_PV' , READ_VAL, NULL))      AS `PM3_STOCK_VALVE_P_PV`
              , GROUP_CONCAT(IF(TAG_ID = '320FIA7505:av' , READ_VAL, NULL))             AS `320FIA7505:av`
              , GROUP_CONCAT(IF(TAG_ID = 'PM3_AFTERDRYER_PV' , READ_VAL, NULL))         AS `PM3_AFTERDRYER_PV`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7514:me' , READ_VAL, NULL))             AS `320PIC7514:me`
              , GROUP_CONCAT(IF(TAG_ID = '320PIC7513:me' , READ_VAL, NULL))             AS `320PIC7513:me`
         FROM (
                SELECT 'STEAM' AS TAG_ID
                     , TOT_DTTM AS READ_DTTM
                     , SUM(TOT_VAL) AS READ_VAL
                 FROM EMS_ECC_TOT_DATA
                WHERE COM_ID = 'MRPAP'
                  AND TOT_DTTM > DATE_SUB(STR_TO_DATE('20221220180000','%Y%m%d%H%i%s'), INTERVAL 450 MINUTE)    /*'20221220180000' 현재시각 변수처리*/
                  AND TOT_DTTM <= STR_TO_DATE('20221220180000','%Y%m%d%H%i%s')                                  /*'20221220180000' 현재시각 변수처리*/
                  AND ECC_ID = 'ECC000009'        /* FIXED */
                  AND READ_OBJ_ID = 'ROI000003'   /* FIXED */
                GROUP BY TOT_DTTM
                UNION ALL
                SELECT T.TAG_ID
                     , T.READ_DTTM
                     , T.READ_VAL
                  FROM EMS_TAG_READ_RAW_DATA T
                 INNER JOIN EMS_ENRG_FCST_VAR V
                    ON T.COM_ID = V.COM_ID
                   AND T.TAG_ID = V.TAG_ID
                   AND V.FCST_ID = '1'          /* FIXED */
                 WHERE T.COM_ID = 'MRPAP'       /* FIXED */
                   AND T.READ_DTTM > DATE_SUB(STR_TO_DATE('20221220180000','%Y%m%d%H%i%s'), INTERVAL 450 MINUTE)
                   AND T.READ_DTTM <= STR_TO_DATE('20221220180000','%Y%m%d%H%i%s')
              ) A
         GROUP BY READ_DTTM
        ) A
  WHERE STEAM >= 0
  ORDER BY DateTime"""
 
# query를 DB 서버에 전달
cur.execute(select_query)

# 배열 형식으로 다 읽어와
resultset = cur.fetchall()

# dataframe 형으로 변환
raw = pd.DataFrame(resultset, columns=['DateTime'
      , 'STEAM'
      , '320FIA7505:av'
      , '320LIC7525:con'
      , '320LIC7526:con'
      , '320LIC7527:con'
      , '320LIC7527:me'
      , '320LIC7528:con'
      , '320LIC7537:con'
      , '320LIC7537:me'
      , '320PIC7509:me'
      , '320PIC7510:con'
      , '320PIC7510:me'
      , '320PIC7511:con'
      , '320PIC7511:me'
      , '320PIC7533:con'
      , '320PIC7533:me'
      , '320PIC7544:con'
      , '320PIC7544:me'
      , '320TIC7540:me'
      , '320TIC7545:con'
      , '320TIC7545:me'
      , 'PM3_AFTERDRYER_OP'
      , 'PM3_ASH_FLOW_OP'
      , 'PM3_ASH_FLOW_PV'
      , 'PM3_ASH_PV'
      , 'PM3_BASIS_WEIGHT_PV'
      , 'PM3_BASIS_WEIGHT_PV2'
      , 'PM3_CW_PV'
      , 'PM3_MOISTURE_PV'
      , 'PM3_MOISTURE_PV2'
      , 'PM3_PREDRYER_OP'
      , 'PM3_PREDRYER_PV'
      , 'PM3_STOCK_CONSISTENCY_PV'
      , 'PM3_STOCK_FLOW_PV'
      , 'PM3_STOCK_VALVE_P_PV'
      , 'PM3_AFTERDRYER_PV'
      , '320PIC7514:me'
      , '320PIC7513:me'])



raw.head(5)

,DateTime,STEAM,320FIA7505:av,320LIC7525:con,320LIC7526:con,320LIC7527:con,320LIC7527:me,320LIC7528:con,320LIC7537:con,320LIC7537:me,...,PM3_MOISTURE_PV,PM3_MOISTURE_PV2,PM3_PREDRYER_OP,PM3_PREDRYER_PV,PM3_STOCK_CONSISTENCY_PV,PM3_STOCK_FLOW_PV,PM3_STOCK_VALVE_P_PV,PM3_AFTERDRYER_PV,320PIC7514:me,320PIC7513:me
0,2022-12-20 10:45:00,5.85156,19.73626,42.82878,71.08994,6.70575,54.60318,20.93174,0.00000,39.82906,...,1.32796,4.54941,235108.79688,1.06900,3.76800,7.01900,47.13000,1.27400,1.27827,0.63626
1,2022-12-20 11:00:00,5.75000,22.18315,40.92590,73.13108,6.97931,55.36020,26.59561,0.00000,41.05006,...,1.48459,4.57566,299438.21875,1.01500,3.79000,7.06700,47.55000,1.28600,1.29170,0.65238
2,2022-12-20 11:15:00,5.85156,21.30403,44.11152,76.22473,6.78766,54.70086,50.91642,0.00000,42.07570,...,1.57253,4.49127,367157.87500,1.02400,3.78800,7.00900,47.18000,1.33900,1.29573,0.65238
3,2022-12-20 11:30:00,5.84766,21.24542,44.02749,73.50533,7.01727,55.28694,52.80920,0.00000,36.87424,...,1.46636,4.46810,435812.75000,1.01900,3.80000,7.02600,47.37000,1.28600,1.29573,0.65775
4,2022-12-20 11:45:00,5.85156,20.55678,45.74142,76.05918,7.04147,55.40904,22.20047,0.00000,25.93407,...,1.49084,4.43820,505214.03125,0.99900,3.79900,6.98800,47.03000,1.28000,1.29707,0.64298


In [11]:
## raw 테이블 Datetime 제외 모두 float으로 변경

# 날짜 컬럼을 index로 넘겨놓고
raw.set_index('DateTime',inplace=True)

# 모든 컬럼 float으로 변경
raw = raw.astype('float')

# index 리셋
raw.reset_index(inplace=True)

raw.dtypes

DateTime                    datetime64[ns]
STEAM                              float64
320FIA7505:av                      float64
320LIC7525:con                     float64
320LIC7526:con                     float64
320LIC7527:con                     float64
320LIC7527:me                      float64
320LIC7528:con                     float64
320LIC7537:con                     float64
320LIC7537:me                      float64
320PIC7509:me                      float64
320PIC7510:con                     float64
320PIC7510:me                      float64
320PIC7511:con                     float64
320PIC7511:me                      float64
320PIC7533:con                     float64
320PIC7533:me                      float64
320PIC7544:con                     float64
320PIC7544:me                      float64
320TIC7540:me                      float64
320TIC7545:con                     float64
320TIC7545:me                      float64
PM3_AFTERDRYER_OP                  float64
PM3_ASH_FLO

#### 2) Process

In [12]:
processed = MM.processing_raw(raw)
processed

,320FIA7505:av,320LIC7525:con,320LIC7526:con,320LIC7527:con,320LIC7527:me,320LIC7528:con,320LIC7537:con,320LIC7537:me,320PIC7509:me,320PIC7510:con,...,PM3_STOCK_FLOW_PV,PM3_STOCK_VALVE_P_PV,PM3_AFTERDRYER_PV,320PIC7514:me,320PIC7513:me,mon,day,hour,min,weekday
0,19.73626,42.82878,71.08994,6.70575,54.60318,20.93174,0.00000,39.82906,0.38278,56.96500,...,7.019,47.13,1.274,1.27827,0.63626,12,20,10,45,1
1,22.18315,40.92590,73.13108,6.97931,55.36020,26.59561,0.00000,41.05006,0.25885,58.89117,...,7.067,47.55,1.286,1.29170,0.65238,12,20,11,0,1
2,21.30403,44.11152,76.22473,6.78766,54.70086,50.91642,0.00000,42.07570,0.23871,58.83220,...,7.009,47.18,1.339,1.29573,0.65238,12,20,11,15,1
3,21.24542,44.02749,73.50533,7.01727,55.28694,52.80920,0.00000,36.87424,0.23871,58.89686,...,7.026,47.37,1.286,1.29573,0.65775,12,20,11,30,1
4,20.55678,45.74142,76.05918,7.04147,55.40904,22.20047,0.00000,25.93407,0.23993,59.06968,...,6.988,47.03,1.280,1.29707,0.64298,12,20,11,45,1
5,21.61172,43.04726,73.48350,7.19315,55.65324,18.06180,0.00000,29.13309,0.22161,59.05314,...,6.998,47.08,1.327,1.29841,0.65104,12,20,12,0,1
6,21.17216,44.35561,74.74279,6.77154,54.62759,39.63893,0.00000,30.64713,0.19536,59.06399,...,7.032,47.35,1.327,1.29035,0.65238,12,20,12,15,1
7,21.96337,45.43290,74.89897,7.23798,55.97070,53.19087,0.00000,30.45177,0.16178,58.92748,...,7.022,47.50,1.273,1.29438,0.64835,12,20,12,30,1
8,31.20879,43.69821,72.91251,6.98238,55.23809,27.67017,1.28167,50.13431,0.10684,63.57915,...,8.297,52.43,1.296,1.27827,0.63761,12,20,12,45,1
9,24.16117,47.85724,77.54648,6.77442,54.62759,17.49062,0.00000,28.05861,0.19414,54.74131,...,8.171,51.82,1.316,1.27692,0.64701,12,20,13,0,1


#### 3) Inference

모듈에서 사용할 모델 지정 후 구동 (XGBoost or LightGBM)

In [13]:
prediction = MM.pred_steamusage(raw, processed)
prediction

c:\Users\limjh\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:173: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


,DateTime,Y_pred
0,2022-12-20 11:00:00,5.798175
1,2022-12-20 11:15:00,6.203488
2,2022-12-20 11:30:00,6.032794
3,2022-12-20 11:45:00,5.954176
4,2022-12-20 12:00:00,5.919655
5,2022-12-20 12:15:00,6.132352
6,2022-12-20 12:30:00,6.029551
7,2022-12-20 12:45:00,6.209529
8,2022-12-20 13:00:00,8.230721
9,2022-12-20 13:15:00,6.673774


#### 4) Comparison

In [31]:
result = MM.compare_real_pred(raw, prediction)
result

,DateTime,Y_real,Y_pred
0,2022-01-01 00:15:00,8.84961,NaN
1,2022-01-01 00:30:00,NaN,9.124606



#### 5) Storage

In [14]:
## 추가 query
insert_query = "insert into ems_enrg_fcst_rslt \
    (COM_ID, FCST_ID, FCST_DTTM, FCST_VAL, REG_USER_NO, REG_DTTM, PROC_USER_NO, PROC_DTTM) \
    values ('MRPAP','1',%s, %s,'AI',sysdate(),'AI',sysdate())"

# 'MRPAP','1','2022-12-22 18:15:00','5','AI',sysdate(),'AI',sysdate()

## for문으로 3번 결과 테이블 행길이만큼 insert반복
row = len(prediction)

for n in range(row):
    cur.execute(insert_query, (prediction.iloc[n,0],prediction.iloc[n,1]))
    conn.commit()


IntegrityError: (1062, "Duplicate entry 'MRPAP-1-2022-12-20 11:00:00' for key 'PRIMARY'")

In [20]:
# Close Connection
conn.close()